<a href="https://colab.research.google.com/github/domschl/torch-poet/blob/master/torch_poet_using_indie_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import shutil
from enum import Enum
import re
import time
import logging
import sys
import json
import random
import torch
import torch.nn as nn
from torch import Tensor

In [2]:
pip install -U ml-indie-tools

In [3]:
from ml_indie_tools.env_tools import MLEnv
from ml_indie_tools.Gutenberg_Dataset import Gutenberg_Dataset
from ml_indie_tools.Text_Dataset import Text_Dataset

# 0. System configuration

This notebook can either run on a local hardware, a jupyter server, or on Google Colab.

This version of the notebook uses [ml-indie-tools](https://github.com/domschl/ml-indie-tools) to detect hardware, persistence handling, and access to training data. 

In [6]:
ml_env=MLEnv(platform='pt')  # use PyTorch
ml_env.describe()

'OS: Linux, Python: 3.7.12, Colab Jupyter Notebook Pytorch: 1.10.0+cu111, CPU'

In [7]:
root_path, project_path, model_path, data_path, log_path = ml_env.init_paths(project_name='women_writers', model_name='lstm_v1')

Mounted at /content/drive


# 1. Text data collection

**Important note:** the following `project_name` determines the root directory for training data and model snapshots, so it should be changed whenever datasets of model configurations are changed.

## 1.1 Project Gutenberg data source

Search, filter, clean and download books from Project Gutenberg

In [8]:
logging.basicConfig(level=logging.INFO)

In [9]:
gd = Gutenberg_Dataset()

In [ ]:
# Get the list of available books on Gutenberg.
gbl=GutenbergLib(cache_dir=os.path.join(root_path, 'gutenberg_cache'))
gbl.load_index()

INFO:GutenbergLib:Gutenberg index read from /content/drive/My Drive/gutenberg_cache/gutenberg_index


In [15]:
# sample searches
search_specs=[
    {"author": ["brontë","Jane Austen", "Woolf"], "language": ["english"]},
]
book_list=gd.search(search_spec)
book_cnt = len(book_list)
print(f"{book_cnt} matching books found with search {search_spec}.")
if book_cnt<40:
    # Note: please verify that book_cnt is 'reasonable'. If you plan to use a large number of texts, 
    # consider [mirroring Gutenberg](https://github.com/domschl/ml-indie-tools#working-with-a-local-mirror-of-project-gutenberg)
    book_list = gd.insert_book_texts(book_list, download_count_limit=book_cnt)  
else:
    logging.error("Please verify your book_list, a large number of books is scheduled for download. ABORTED.")

INFO:GutenbergLib:Book read from cache at gutenberg/65754.txt
INFO:GutenbergLib:Book read from cache at gutenberg/64457.txt
INFO:GutenbergLib:Book read from cache at gutenberg/63022.txt
INFO:GutenbergLib:Book read from cache at gutenberg/60627.txt
INFO:GutenbergLib:Book read from cache at gutenberg/60239.txt
INFO:GutenbergLib:Book read from cache at gutenberg/55679.txt
INFO:GutenbergLib:Book read from cache at gutenberg/54066.txt
INFO:GutenbergLib:Book read from cache at gutenberg/54012.txt
INFO:GutenbergLib:Book read from cache at gutenberg/54011.txt
INFO:GutenbergLib:Book read from cache at gutenberg/54010.txt
INFO:GutenbergLib:Book read from cache at gutenberg/53747.txt
INFO:GutenbergLib:Book read from cache at gutenberg/42671.txt
INFO:GutenbergLib:Book read from cache at gutenberg/42078.txt
INFO:GutenbergLib:Book read from cache at gutenberg/36285.txt
INFO:GutenbergLib:Book read from cache at gutenberg/31100.txt


30 matching books found with search {'author': ['brontë', 'Jane Austen', 'Woolf'], 'language': ['english']}.


INFO:GutenbergLib:Book read from cache at gutenberg/9182.txt
INFO:GutenbergLib:Book read from cache at gutenberg/5670.txt
INFO:GutenbergLib:Book read from cache at gutenberg/1342.txt
INFO:GutenbergLib:Book read from cache at gutenberg/1245.txt
INFO:GutenbergLib:Book read from cache at gutenberg/1212.txt
INFO:GutenbergLib:Book read from cache at gutenberg/946.txt
INFO:GutenbergLib:Book read from cache at gutenberg/771.txt
INFO:GutenbergLib:Book read from cache at gutenberg/768.txt
INFO:GutenbergLib:Book read from cache at gutenberg/767.txt
INFO:GutenbergLib:Book read from cache at gutenberg/161.txt
INFO:GutenbergLib:Book read from cache at gutenberg/158.txt
INFO:GutenbergLib:Book read from cache at gutenberg/144.txt
INFO:GutenbergLib:Book read from cache at gutenberg/141.txt
INFO:GutenbergLib:Book read from cache at gutenberg/121.txt
INFO:GutenbergLib:Book read from cache at gutenberg/105.txt


## 1.2 Text library

`TextLibrary` class: text library for training, encoding, batch generation,
and formatted source display. It read some books from Project Gutenberg
and supports creation of training batches. The output functions support
highlighting to allow to compare generated texts with the actual sources
to help to identify identical (memorized) parts of a given length.

In [18]:
td = Text_Dataset(book_list)

INFO:Datasets:Loaded 30 texts


In [19]:
class TextLibraryDataset(torch.utils.data.Dataset):
    def __init__(self, text_dataset, sample_length, torch_device, text_stepping=10):
        self.device=torch_device
        self.text_length=0
        full_text=""
        for text in text_dataset.textlist:
            if 'text' in text:
                full_text += text['text']
        self.text_length = len(full_text)
        text_encode = text_dataset.encode(full_text, tokenizer='char')
        self.vocab_size = len(text_dataset.i2c)
        self.text_stepping=text_stepping
        self.sample_length=sample_length
        self.records=int((self.text_length-sample_length-1)/text_stepping)
        self.data=torch.LongTensor(text_encode).to(self.device)
        del text_encode
        del full_text
        
    def __len__(self):
        return self.records

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if idx>=self.length:
            return None
        X=self.data[idx*self.text_stepping:idx*self.text_stepping+self.sample_length].to(self.device)
        y=self.data[idx*self.text_stepping+1:idx*self.text_stepping+self.sample_length+1].to(self.device)
        return X,y

# 2. The deep LSTM model

# 2.1 Model configuration parameters

In [ ]:
model_params = {
    "model_name": libdesc['name'],
    "vocab_size": len(textlib.i2c),
    "neurons": 768,
    "layers": 8,
    "learning_rate": 1e-4,
    "steps": 80,
    "batch_size": 768
}

## 2.2 The char-rnn model class

In [ ]:
class Poet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(Poet, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.device=device
       
        self.oh = torch.eye(input_size, device=self.device)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0)
        
        self.demb = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=-1)  # negative dims are a recent thing (as 2018-03), remove for old vers.
    
    def init_hidden(self, batch_size):
        self.h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=self.device)
        self.c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=self.device)

    def forward(self, inputx, steps):
        lstm_input=self.oh[inputx]
        self.lstm.flatten_parameters()
        hn, (self.h0, self.c0) = self.lstm(lstm_input, (self.h0, self.c0))
        hnr = hn.contiguous().view(-1,self.hidden_size)
        op = self.demb(hnr)
        opr = op.view(-1, steps ,self.output_size)
        return opr

    def generate(self, n, start=None, temperature=1.0):
        s=''
        torch.set_grad_enabled(False)
        if start==None or len(start)==0:
            start=' '
        self.init_hidden(1)
        for c in start:
            Xt=torch.LongTensor([[textlib.c2i[c]]])
            ypl = self.forward(Xt,1)
            ypl2 = ypl.view(-1,self.output_size)
            if temperature>0.0:
                ypl2 = ypl2 / temperature
            yp = self.softmax(ypl2)
        for i in range(n):
            ypc=Tensor.cpu(yp.detach()) # .cpu()
            y_pred=ypc.numpy()
            inds=list(range(self.output_size))
            ind = np.random.choice(inds, p=y_pred.ravel())
            s=s+textlib.i2c[ind]
            X=np.array([[ind]])
            Xt=torch.LongTensor(X)
            ypl = self.forward(Xt,1)
            ypl2 = ypl.view(-1,self.output_size)
            if temperature>0.0:
                ypl2 = ypl2 / temperature
            yp = self.softmax(ypl2)
        torch.set_grad_enabled(True)
        return s    

## 2.3 Model instance

In [ ]:
poet = Poet(model_params['vocab_size'], model_params['neurons'], model_params['layers'], model_params['vocab_size'], device).to(device)

## 2.4 Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
opti = torch.optim.Adam(poet.parameters(),lr=model_params['learning_rate']);

## 2.5 Helper Functions

These allow to save or restore the training data. Saving and restoring can either be performed:

* Jupyter: store/restore in a local directory,
* Colab: store/restore on google drive. The training-code (using load_checkpoint()) will display an authentication url and code input-box in order to be able to access your google drive from this notebook. This allows to continue training sessions (or inference) after the Colab session was terminated.

In [ ]:
if is_colab_notebook:
    if colab_google_drive_snapshots is True:
        snapshot_path=os.path.join(root_path,f"Colab Notebooks/{model_params['model_name']}/Snapshots")
    else:
        snapshot_path=None
else:
    snapshot_path=os.path.join(root_path,f"{model_params['model_name']}/Snapshots")

In [ ]:
def get_project_path():
    if snapshot_path is None:
        return None
    project_path_ext=f"model-{model_params['vocab_size']}x{model_params['steps']}x{model_params['layers']}x{model_params['neurons']}"
    return os.path.join(snapshot_path, project_path_ext)

def create_project_path():
    if snapshot_path is None:
        return None
    ppath=get_project_path()
    pathlib.Path(ppath).mkdir(parents=True, exist_ok=True)

In [ ]:
if snapshot_path is not None:
    pathlib.Path(snapshot_path).mkdir(parents=True, exist_ok=True)
    create_project_path()
    with open(os.path.join(get_project_path(),'model_params.json'),'w') as f:
        json.dump(model_params,f,indent=4)

In [ ]:
def save_checkpoint(epoch, loss, pr, best_pr, filename='checkpoint.pth.tar'):
    if snapshot_path is None:
        return
    state={
            'epoch': epoch,
            'model_config': model_params,
            'state_dict': poet.state_dict(),
            'optimizer' : opti.state_dict(),
            'precision': pr,
            'loss': loss,
        }
    project_path=get_project_path()
    save_file=os.path.join(project_path,filename)
    best_file=os.path.join(project_path,'model_best.pth.tar')
    torch.save(state, save_file)
    if pr >= best_pr:
        shutil.copyfile(save_file, best_file )
        print(f"Saved best precision model, prec={pr:.3f}")
    else:
        print(f"Saved last model data, prec={pr:.3f}")

def save_history(history, filename="history.json"):
    if snapshot_path is None:
        return
    project_path=get_project_path()
    save_file=os.path.join(project_path,filename)
    try:
        with open(save_file, 'w') as f:
            json.dump(history, f)
    except Exception as e:
        print(f"Failed to write training history file {save_file}, {e}")

def load_history(filename="history.json"):
    if snapshot_path is None:
        return [], time.time()
    project_path=get_project_path()
    load_file=os.path.join(project_path,filename)
    try:
        with open(load_file, 'r') as f:
            history=json.load(f)
    except Exception as e:
        print(f"Starting new history file {load_file}")
        return [], time.time()
    if len(history)>0:
        start=history[-1]["timestamp"]
    return history, start

def load_checkpoint(filename='checkpoint.pth.tar'):
    if snapshot_path is None:
        return 0,0
    project_path=get_project_path()
    load_file=os.path.join(project_path,filename)
    if not os.path.exists(load_file):
        print(load_file)
        print("No saved state, starting from scratch.")
        return 0,0
    state=torch.load(load_file)
    mod_conf = state['model_config']
    for param in ['model_name', 'learning_rate', 'batch_size']:
        if mod_conf[param]!=model_params[param]:
            print(f"Warning: project {param} has changed from {mod_conf[param]} to {model_params[param]}")
            mod_conf[param]=model_params[param]
    if model_params!=mod_conf:
        print(f"The saved model has an incompatible configuration than the current model: {mod_conf} vs. {model_params}")
        print("Cannot restore state, starting from scratch.")
        return 0,0
    poet.load_state_dict(state['state_dict'])
    opti.load_state_dict(state['optimizer'])
    epoch = state['epoch']
    loss = state['loss']
    best_pr = state['precision']
    print(f"Continuing from saved state epoch={epoch+1}, loss={loss:.3f}")  # Save is not necessarily on epoch boundary, so that's approx.
    return epoch,loss

# 3. Training

If there is already saved training data, this step is optional, and alternatively, ch. 4 can be continued.

## 3.1 Training helpers

In [ ]:
def torch_data_loader(batch_size, sample_length, device):
    textlib_dataset=TextLibraryDataset(textlib,sample_length, device)
    data_loader=torch.utils.data.DataLoader(textlib_dataset,batch_size=batch_size, shuffle=True, num_workers=0)
    return data_loader

# Get one sample:
# X, y = next(iter(data_loader))

def precision(y, yp):
    return (torch.sum(yp==y)/float((y.size()[0]*y.size()[1]))).item()

def train(Xt, yt):
    poet.zero_grad()

    poet.init_hidden(Xt.size(0))
    output = poet(Xt, model_params['steps'])
    
    olin=output.view(-1,model_params['vocab_size'])
    _, ytp=torch.max(output,2)
    ytlin=yt.view(-1)

    pr=precision(yt,ytp)
            
    loss = criterion(olin, ytlin)
    ls = loss.item()
    loss.backward()
    opti.step()

    return ls, pr

## 3.2 The actual training loop

In [ ]:
ls=0
nr=0
prs=0
# torch.cuda.empty_cache()
create_project_path()
epoch_start, _ = load_checkpoint()
history, start_time = load_history()
pr=0.0
best_pr=0.0

data_loader=torch_data_loader(model_params['batch_size'], model_params['steps'], device)
    
# Make a snapshot of the trained parameters every snapshot_interval_sec
snapshot_interval_sec=180
# Generate text samples every sample_intervall_sec
sample_interval_sec=600

last_snapshot=time.time()
last_sample=time.time()

bench_all=0
bench_data=0
bench_train=0
bench_sample=0
bench_snapshot=0
bench_output_times=10  # Give 10 benchmark outputs, then stop (it stays more or less same)
sample_train_time=0

for e in range(epoch_start,2500000):
    t1=time.time()
    t0=time.time()
    for Xi,yi in data_loader:
        t2=time.time()
        # this cannot be done in data_loader, if multiprocessing is used :-/
        Xt=Xi #.to(device)
        yt=yi #.to(device)
        
        Xt.requires_grad_(False)
        yt.requires_grad_(False)

        bench_data += time.time()-t1
        t1=time.time()
        l, pr = train(Xt,yt)
        bench_train += time.time()-t1

        ls=ls+l
        prs=prs+pr
        nr=nr+1
        cur_loss=ls/nr
        cur_pr=prs/nr
        if time.time()-last_snapshot > snapshot_interval_sec:
            t1=time.time()
            nr=0
            ls=0
            prs=0
            if cur_pr>best_pr:
                best_pr=cur_pr
            last_snapshot=time.time()
            print(f"Epoch {e+1} Loss: {cur_loss:.3f} Precision: {cur_pr:.3f} Time/Sample: {sample_train_time:.6f} sec/sample")
            save_checkpoint(e,cur_loss,cur_pr, best_pr)
            # if use_cuda:
            #     print(f"Cuda memory allocated: {torch.cuda.memory_allocated()} max_alloc: {torch.cuda.max_memory_allocated()} cached: {torch.cuda.memory_cached()} max_cached: {torch.cuda.max_memory_cached()}")
            hist={"epoch": e+1, "loss": cur_loss, "precision": cur_pr, "timestamp": time.time()-start_time}
            history.append(hist)
            save_history(history)
            bench_snapshot+=time.time()-t1

            if bench_all > 0 and bench_output_times>0:
                bd=bench_data/bench_all*100.0
                bt=bench_train/bench_all*100.0
                bs=bench_sample/bench_all*100.0
                bss=bench_snapshot/bench_all*100.0
                bo=(bench_all-bench_data-bench_train-bench_sample-bench_snapshot)/bench_all*100.0
                print(f"Profiling: data-loading: {bd:.2f}%, training: {bt:.2f}%, sample gen: {bs:.2f}%, snapshots: {bss:.2f}%, overhead: {bo:.2f}%")
                bench_output_times = bench_output_times - 1
                
        sample_train_time=(time.time()-t2)/len(Xt)

        if time.time()-last_sample > sample_interval_sec and cur_loss<1.5:
            t1=time.time()
            last_sample=time.time()
            for temperature in [0.6, 0.7, 0.8]:
                print(f"Temperature {temperature}:")
                tgen=poet.generate(700,". ", temperature=temperature)
                textlib.source_highlight(tgen,minQuoteSize=10,dark_mode=use_dark_mode,display_ref_anchor=False)
            bench_sample+=time.time()-t1
        t1=time.time()
        bench_all+=time.time()-t0
        t0=time.time()


Continuing from saved state epoch=41, loss=0.921
Epoch 41 Loss: 0.918 Precision: 0.714 Time/Sample: 0.002215 sec/sample
Saved best precision model, prec=0.714
Profiling: data-loading: 0.77%, training: 100.18%, sample gen: 0.00%, snapshots: 2.61%, overhead: -3.56%
Epoch 41 Loss: 0.918 Precision: 0.714 Time/Sample: 0.002190 sec/sample
Saved best precision model, prec=0.714
Profiling: data-loading: 0.71%, training: 98.46%, sample gen: 0.00%, snapshots: 2.61%, overhead: -1.78%
Epoch 41 Loss: 0.918 Precision: 0.714 Time/Sample: 0.002206 sec/sample
Saved last model data, prec=0.714
Profiling: data-loading: 0.68%, training: 97.90%, sample gen: 0.00%, snapshots: 2.17%, overhead: -0.74%
Temperature 0.6:


Temperature 0.7:


Temperature 0.8:


Epoch 41 Loss: 0.918 Precision: 0.714 Time/Sample: 0.002202 sec/sample
Saved best precision model, prec=0.714
Profiling: data-loading: 0.65%, training: 95.62%, sample gen: 2.33%, snapshots: 2.27%, overhead: -0.88%
Epoch 41 Loss: 0.920 Precision: 0.714 Time/Sample: 0.002200 sec/sample
Saved last model data, prec=0.714
Profiling: data-loading: 0.64%, training: 95.87%, sample gen: 1.87%, snapshots: 2.07%, overhead: -0.44%
Epoch 41 Loss: 0.919 Precision: 0.714 Time/Sample: 0.002186 sec/sample
Saved last model data, prec=0.714
Profiling: data-loading: 0.63%, training: 96.25%, sample gen: 1.55%, snapshots: 1.95%, overhead: -0.38%
Temperature 0.6:


Temperature 0.7:


...

Temperature 0.7:


Temperature 0.8:


Epoch 49 Loss: 0.915 Precision: 0.714 Time/Sample: 0.002201 sec/sample
Saved last model data, prec=0.714
Epoch 49 Loss: 0.914 Precision: 0.715 Time/Sample: 0.002212 sec/sample
Saved last model data, prec=0.715
Epoch 49 Loss: 0.915 Precision: 0.715 Time/Sample: 0.002201 sec/sample
Saved last model data, prec=0.715
Temperature 0.6:


Temperature 0.7:


Temperature 0.8:


# 4. Text generation

## 4.1 Sample generation

In [ ]:
load_checkpoint(filename="model_best.pth.tar")

Continuing from saved state epoch=41, loss=0.919


(40, 0.9189830598377046)

In [ ]:
def detectPlagiarism(generatedtext, textlibrary, minQuoteLength=10, display_ref_anchor=True):
    textlibrary.source_highlight(generatedtext, minQuoteSize=minQuoteLength,dark_mode=use_dark_mode, display_ref_anchor=display_ref_anchor)

In [ ]:
print("Sample text:")
print("")
for temperature in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    tgen=poet.generate(1000,"\n\n", temperature=temperature)
    print(f"================Temperature: {temperature}==============")
    detectPlagiarism(tgen, textlib, display_ref_anchor=False)

Sample text:

================Temperature: 0.2==============


================Temperature: 0.3==============


================Temperature: 0.4==============


================Temperature: 0.5==============


================Temperature: 0.6==============


================Temperature: 0.7==============


================Temperature: 0.8==============


================Temperature: 0.9==============


================Temperature: 1.0==============


## 4.2 Dialog with the model

In [ ]:
# Do a dialog with the recursive neural net trained above:
def doDialog():
    temperature = 0.6  # 0.1 (free-style chaos) - >1.0 (rigid, frozen)
    endPrompt = '.'  # the endPrompt character is the end-mark in answers.
    numSentences = 3 # Try to generate numSentences terminated by endPrompt
    # maxEndPrompts = 4  # look for number of maxEndPrompts until answer is finished.
    # maxAnswerSize = 2048  # Maximum length of the answer
    # minAnswerSize = 64  # Minimum length of the answer

    
    print("Please enter some dialog.")
    print("The net will answer according to your input.")
    print("'bye' for end,")
    print("'reset' to reset the conversation context,")
    print("'temperature=<float>' [0.1(free, chaotic) - >1.0(strict, frozen)]")
    print("    to change character of the dialog.")
    # print("    Current temperature={}.".format(temperature))
    print()
    xso = None
    bye = False
    last_ans=""
        
    while not bye:
        print("> ", end="")
        prompt = input()
        if prompt == 'bye':
            bye = True
            print("Good bye!")
            continue
        if prompt.find("temperature")>=0 and prompt.find("=") > prompt.find("temperature"):
            temperature=float(prompt[prompt.find('=')+1:])
            print(f"Temperature set to {temperature}")
            continue

        prompt+=' '
        for attempts in range(0,3):
            tgen=poet.generate(2000,prompt,temperature=temperature)
            # tgen=tgen.replace("Mr.", "Mr")
            # tgen=tgen.replace("Mrs.", "Mrs")
            # tgen=tgen.replace("\n"," ")
            # tgen=tgen.replace("  "," ")
            tgi=tgen.split(". ")
            print(f"{len(tgi)} sentences")
            if len(tgi)<numSentences:
                continue
            ans=""
            for i in range(0,numSentences):
                ans += tgi[i]+". "
            break
            # i=tgen.find(endPrompt)
            # i2=tgen[i+1:].find(endPrompt)+i
            # i3=tgen[i2+1:].find(endPrompt)+i2
            # i4=tgen[i3+1:].find(endPrompt)+i3
            # tgen=tgen[i+1:i4+2]
            # if len(tgen)>10:
            #     break
        last_ans=ans
        textlib.source_highlight(last_ans, minQuoteSize=10,dark_mode=use_dark_mode,display_ref_anchor=False)
    return

In [ ]:
doDialog()

Please enter some dialog.
The net will answer according to your input.
'bye' for end,
'reset' to reset the conversation context,
'temperature=<float>' [0.1(free, chaotic) - >1.0(strict, frozen)]
    to change character of the dialog.

> The interaction between mind and body is governed by a soul.
3 sentences


> A proof for the existence of god.
3 sentences


> Ein Beweis für die Existenz Gottes.
4 sentences


> Materie und Geist sind von einer Substanz
9 sentences


> Das Ding an sich ist dem Menschen nicht zugänglich. 
6 sentences


> Human life as the expression of freedom.
5 sentences


> 

KeyboardInterrupt: ignored